# Shape Mode Decomposition-Batch Analysis

Use this notebook to calculate the following for a batch of simulations

Results:
- Filament shape modes
- Mode amplitude of the filament for each shape mode vs time
- PCA dimensionality estimate using participation ratio

In [6]:
import sys
if 'init_modules' in globals().keys():
    # second or subsequent run: remove all but initially loaded modules
    for m in sys.modules.keys():
        if m not in init_modules:
            del(sys.modules[m])
else:
    # first run: find out which modules were initially loaded
    init_modules = sys.modules.keys()
    
import numpy as np
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import interpolate

import os
import pyfilaments.analysisutils as analysis
import imp
imp.reload(analysis)
%matplotlib notebook
plt.close('all')

# Figure parameters
from matplotlib import rcParams
from matplotlib import rc
from matplotlib import cm
from tqdm import tqdm

rc('font', family='sans-serif') 
rc('font', serif='Helvetica') 
rc('text', usetex='false') 
rcParams.update({'font.size': 12})

from joblib import Parallel, delayed
import multiprocessing

In [7]:
# batch_data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ModellingResults/Analysis_test'

batch_data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweep_widerICS_FINAL_2021-12-23'

# batch_data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweeps_diffActivityTimeScales_FINAL/activityTime_750'

data_folders = os.listdir(batch_data_folder)

print(len(data_folders))

40


In [8]:
# Batch analysis

# Create file list

full_path_list = []

for folder in tqdm(data_folders):
    
    cwd = os.path.join(batch_data_folder, folder)

    files = [file for file in os.listdir(cwd) if file.endswith('.hdf5') ]
    
    
    for file in files:
        
        full_path_list.append(os.path.join(cwd, file))
        

print(len(full_path_list))



100%|██████████| 40/40 [00:00<00:00, 5416.90it/s]

400


In [9]:
plt.close('all')


def run_pca(full_path = None):

        fil = analysis.analysisTools(file = full_path)

        ic_analysis_folder = os.path.join(fil.rootFolder, 'IC_analysis')

        fil.ic_analysis_folder = ic_analysis_folder
        
        print(fil.Time[-1])

        if(fil.Time[-1] >= 250*fil.activity_timescale):
            # Completed simulation found.
            # Run the PCA analysis
            
            print('completed simulation data found!')
            # Compute tangent angle matrix
            fil.compute_tangent_angle_matrix()

            # Neglect transients
            start_cycle = 100 # Ignore the first 100 cycles to remove transients

            start_time = start_cycle*fil.activity_timescale

            start_index = next((i for i,x in enumerate(fil.Time) if x>= start_time), 0)

            # Ignore the transients in the simulation
            fil.tangent_angles_matrix = fil.tangent_angles_matrix[start_index:,:]


            # Compute the shape Covariance matrix
            fil.compute_shape_covariance_matrix()

            # Eigen-decomposition of the covariance matrix
            d_sig, eigenvectors_sig, d_normalized =\
            fil.matrix_eigen_decomposition(matrix = fil.covariance_matrix)

            # PCA dimension based on partcipation ratio
            pca_dim = fil.compute_participation_ratio()
            
            print(pca_dim)

            # Save the eigenvectors
            fil.save_eigenvectors()

#             fil.plot_shape_modes(save = False)
            fil.project_filament_shapes()
            fil.save_mode_amplitudes()


            found_completed_sim = True

        

In [10]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores, verbose=10)(delayed(run_pca)(file) for file in full_path_list)


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:   10.3s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:   13.2s
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:   24.3s
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   30.0s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:   36.2s
[Parallel(n_jobs=12)]: Done  48 tasks      | elapsed:   43.8s
[Parallel(n_jobs=12)]: Done  61 tasks      | elapsed:   54.4s
[Parallel(n_jobs=12)]: Done  74 tasks      | elapsed:  1.1min
[Parallel(n_jobs=12)]: Done  89 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 104 tasks      | elapsed:  1.5min
[Parallel(n_jobs=12)]: Done 121 tasks      | elapsed:  1.8min
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:  2.0min
[Parallel(n_jobs=12)]: Done 157 tasks      | elapsed:  2.2min
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:  2.4min
[Parallel(n_jobs=12)]: Done 197 tasks      | elapsed:  

In [7]:
for file in tqdm(full_path_list):
    run_pca(file)

  0%|          | 0/60 [00:00<?, ?it/s]

187500.0
completed simulation data found!


  2%|▏         | 1/60 [00:03<03:13,  3.27s/it]

No:of spatial points: 100
No:of time points: 22501
(22501, 100)
(22501, 100)
No:of eigenvalues to explain 95 perc of variance: 3
1.9896095767834021
187500.0
completed simulation data found!


  3%|▎         | 2/60 [00:06<03:10,  3.29s/it]

...

No:of spatial points: 100
No:of time points: 22501
(22501, 100)
(22501, 100)
No:of eigenvalues to explain 95 perc of variance: 4
2.3442856619447388
187500.0
completed simulation data found!


 98%|█████████▊| 59/60 [02:35<00:03,  3.23s/it]

No:of spatial points: 100
No:of time points: 22501
(22501, 100)
(22501, 100)
No:of eigenvalues to explain 95 perc of variance: 4
2.3023024055845114
187500.0
completed simulation data found!


100%|██████████| 60/60 [02:38<00:00,  2.64s/it]

No:of spatial points: 100
No:of time points: 22501
(22501, 100)
(22501, 100)
No:of eigenvalues to explain 95 perc of variance: 4
2.3016503891025226


In [23]:
def run_shape_mode_decomposition(file):
    
    fil = analysis.analysisTools(file = file)

    ic_analysis_folder = os.path.join(fil.rootFolder, 'IC_analysis')

    fil.ic_analysis_folder = ic_analysis_folder

    print(fil.Time[-1])

    if(fil.Time[-1]==250*fil.activity_timescale):
        # Completed simulation found.
        # Run the PCA analysis

        print('completed simulation data found!')
        # Compute tangent angle matrix
        fil.compute_tangent_angle_matrix()

        # Neglect transients
        start_cycle = 0 # Ignore the first 100 cycles to remove transients

        start_time = start_cycle*fil.activity_timescale

        start_index = next((i for i,x in enumerate(fil.Time) if x>= start_time), 0)

        # Ignore the transients in the simulation
        fil.tangent_angles_matrix = fil.tangent_angles_matrix[start_index:,:]

        # Compute the shape Covariance matrix
        fil.compute_shape_covariance_matrix()

        # Eigen-decomposition of the covariance matrix
        d_sig, eigenvectors_sig, d_normalized =\
        fil.matrix_eigen_decomposition(matrix = fil.covariance_matrix)


        # Save the no:of eigvalues necessary to explain 95% of variance to file
#         fil.save_eigenvectors()

    #             fil.plot_shape_modes(save = False)
        fil.project_filament_shapes()
        fil.save_mode_amplitudes()


        found_completed_sim = True

In [13]:
file = '/home/deepak/ActiveFilamentsSearch_backup_3/ActivityStrengthSweeps_diffActivityTimeScales_FINAL/activityTime_750/SimData_Np_32_Shape_line_kappa_hat_6_k_25_b0_2.1_F_0_S_0_D_0.5_activityTime_750_simType_point/sim_data_00_.hdf5'

In [16]:
run_shape_mode_decomposition(file)

187500.0
completed simulation data found!
No:of spatial points: 100
No:of time points: 37501
(37501, 100)
(37501, 100)
No:of eigenvalues to explain 95 perc of variance: 3


In [24]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores, verbose=10)(delayed(run_shape_mode_decomposition)(file) for file in tqdm(full_path_list))

  0%|          | 0/400 [00:00<?, ?it/s][Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=12)]: Batch computation too fast (0.1595s.) Setting batch_size=2.
 12%|█▏        | 48/400 [00:00<00:01, 178.04it/s][Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=12)]: Batch computation too fast (0.1896s.) Setting batch_size=4.
[Parallel(n_jobs=12)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=12)]: Done  50 tasks      | elapsed:    0.6s
100%|██████████| 400/400 [00:02<00:00, 137.82it/s]
[Parallel(n_jobs=12)]: Done 296 tasks      | elapsed:    3.2s
[Parallel(n_jobs=12)]: Done 343 tasks      | elapsed:    3.6s
[Parallel(n_jobs=12)]: Done 377 out of 400 | elapsed:    3.9s remaining:    0.2s
[Parallel(n_jobs=12)]: Done 400 out of 400 | elapsed:    4.0s finished
